In [3]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import shutil as sh

from webdriver_manager.chrome import ChromeDriverManager
import warnings

import pandas as pd
import numpy as np
import seaborn as sns 



In [15]:
alpha = ['a','b','c','d','e','f','g',
         'h','i','j','k','l','m','n','o','p',
        'q','r','s','t','u','v','w','y','z']

player_links = []

chromedriver = "/Applications/chromedriver" 
os.environ["webdriver.chrome.driver"] = chromedriver

In [37]:
def get_player_name_links(player_links,alpha):
    for i in range(len(alpha)):
    
        # opening player names list for each letter in alphabet except z
        url = 'https://www.basketball-reference.com/wnba/players/{}/'.format(alpha[i])
        base_link = 'https://www.basketball-reference.com/'
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        
        driver = webdriver.Chrome(chromedriver)
        driver.get(url)
        driver.page_source[:]
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    
        # finding player stats links
        div = soup.find_all('div',id = 'content')
        paragraph = div[0].find_all('p')
        
        
        
       
        for j in range(len(paragraph)):
            player_stats_link = paragraph[j].find('a').attrs['href'].strip('/')
            if 'html' in player_stats_link:
                player_links.append(base_link+player_stats_link)
    
        driver.close()
    
    return player_links
            
player_name_links = get_player_name_links(player_links,alpha)  

In [44]:
wnba_player_links = []
for i in range(len(player_name_links)):
    if 'players' in player_name_links[i]:
        wnba_player_links.append(player_name_links[i])
        
wnba_player_links[-1]

'https://www.basketball-reference.com/wnba/players/z/zolmash01w.html'

In [119]:
def get_player_stats(player_name_links):
    
    STATS = []
    player_dictionary = {}
    names = []
    for i in range(400):
        url = player_name_links[i]
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        
        driver = webdriver.Chrome(chromedriver)
        driver.get(url)
        driver.page_source[:]
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        table = soup.find_all('tbody')
        iD = 0
        for i in range(len(table)):
            if 'id="advanced0.' in str(table[i].find('tr')):
                iD = i
        table_rows = table[iD].find_all('tr')
    
        table_head = table[iD].find_all('th')
        find_name = soup.find('h1')
        name = find_name.text.strip()[:-11]
        names.append(name)
        year_stats = []
        player_stats = []
        
        
        for i in range(len(table_rows)):
            year = table_head[i].text
            table_data = table_rows[i].find_all('td')
            year_stats = []
            year_stats.append(year)
            for j in range(len(table_data)):
                if table_data[j].attrs['data-stat'] != 'DUMMY':
                    year_stats.append(table_data[j].text)
            player_stats.append(year_stats)
        
        player_dictionary[name] = player_stats
        
        driver.close()
    
    return player_dictionary,names

In [120]:
ALL_PLAYERS_STATS,names = get_player_stats(wnba_player_links) 

In [121]:
# filling empty strings with zero , for type conversion
for key in ALL_PLAYERS_STATS.keys():
    for i in range(len(ALL_PLAYERS_STATS[key])):
        for j in range(len(ALL_PLAYERS_STATS[key][i])):
            if ALL_PLAYERS_STATS[key][i][j] == '':
                ALL_PLAYERS_STATS[key][i][j] = 0

In [122]:
len(ALL_PLAYERS_STATS)

399

In [123]:
# making player data frames 
player_dataframes = []
for key in ALL_PLAYERS_STATS.keys():
    indexA = []
    indexB = []
    for i in range(len(ALL_PLAYERS_STATS[key])):
        indexA.append(key)
        indexB.append(ALL_PLAYERS_STATS[key][i][0])
    if len(ALL_PLAYERS_STATS[key][0]) == 21: 
        arrays = [np.array(indexA),np.array(indexB)]
        stats = pd.DataFrame(ALL_PLAYERS_STATS[key],index=arrays,columns = ['Season','Team','Age',
                'G','MP','PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%',
                'STL%','BLK%','TOV%','USG%','OWS','DWS','WS','WS/48'])
        del stats['Season']
        STATS = stats.astype({'Age':'int','Team':'str','G':'int','MP':'int',
                'PER':'float','TS%':'float','3PAr':'float','FTr':'float',
                'ORB%':'float','DRB%':'float','TRB%':'float','AST%':'float',
                'STL%':'float','BLK%':'float','TOV%':'float','USG%':'float',
                'OWS':'float','DWS':'float','WS':'float','WS/48':'float'})
        player_dataframes.append(STATS)
    else:
        pass

In [9]:
WNBA = pd.concat(player_dataframes)

In [131]:
WNBA

Team  Age   G   MP   PER    TS%   3PAr    FTr  ORB%  DRB%  \
Farhiya Abdi   2013  LAS   21  17  122   8.5  0.429  0.000  0.308   4.1   7.3   
               2014  LAS   22  17  142   5.7  0.463  0.159  0.068   2.6   5.3   
               2015  LAS   23  18  237   5.4  0.420  0.418  0.089   2.8  12.4   
Tajama Abraham 1997  SAC   21  28  422   6.4  0.427  0.000  0.302   9.2  10.8   
               1998  DET   22  12   44   8.2  0.437  0.000  1.071   5.4  13.4   
...                  ...  ...  ..  ...   ...    ...    ...    ...   ...   ...   
Tayler Hill    2017  WAS   26  18  462  17.0  0.531  0.466  0.419   1.2   7.1   
               2018  TOT   27  20  235   4.9  0.410  0.536  0.167   1.5   6.1   
               2018  WAS   27  13  117   6.2  0.440  0.538  0.308   1.0   7.2   
               2018  DAL   27   7  118   3.7  0.381  0.533  0.044   2.0   4.9   
               2019  DAL   28   4   49   9.9  0.428  0.462  0.385   4.6   7.7   

                     TRB%  AST%  STL%  BLK%  TOV%  USG%  OWS  DWS   WS  WS/48  
Farhiya Abdi   2013   5.8  11.2   0.8   0.6   8.3  17.9  0.1  0.1  0.2  0.065  
               2014   3.9   5.7   0.4   0.0  11.7  16.5  0.0  0.0  0.0 -0.008  
               2015   7.8   6.5   1.6   0.7  13.7  19.6 -0.4  0.1 -0.3 -0.055  
Tajama Abraham 1997  10.0   6.0   1.5   2.3  25.6  20.3 -0.6 -0.2 -0.8 -0.092  
               1998   9.4   0.0   2.5   1.8  19.5  26.4 -0.1  0.1  0.0 -0.001  
...                   ...   ...   ...   ...   ...   ...  ...  ...  ...    ...  
Tayler Hill    2017   4.2  18.3   1.8   0.2  10.3  24.5  1.3  0.2  1.5  0.155  
               2018   3.8  12.8   0.7   0.7  15.9  20.3 -0.4  0.0 -0.4 -0.083  
               2018   4.1  16.6   0.9   0.7  19.9  21.7 -0.2  0.0 -0.2 -0.072  
               2018   3.4   9.1   0.4   0.7  11.6  18.9 -0.2  0.0 -0.2 -0.094  
               2019   6.1  10.8   1.1   0.0   0.0  14.3  0.1  0.0  0.1  0.077  

[1616 rows x 20 columns]

In [4]:
WNBA.to_pickle("WNBA")

In [13]:
from sqlalchemy import create_engine

# pickling in data 
WNBA = pd.read_pickle('WNBA')

# instantiating engine
engine = create_engine('sqlite:///WNBASTATS.db', echo=True)

# establishing connection to SQLite
sqlite_connection = engine.connect()

2021-06-25 05:45:17,291 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-06-25 05:45:17,292 INFO sqlalchemy.engine.base.Engine ()
2021-06-25 05:45:17,293 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-06-25 05:45:17,295 INFO sqlalchemy.engine.base.Engine ()


In [14]:
# Reading in WrkSet DataFrame into a Table named "Stats" in SQLlite
sqlite_table = "AdvStatsW"
WNBA.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-06-25 05:45:20,441 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("AdvStatsW")
2021-06-25 05:45:20,442 INFO sqlalchemy.engine.base.Engine ()
2021-06-25 05:45:20,444 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("AdvStatsW")
2021-06-25 05:45:20,445 INFO sqlalchemy.engine.base.Engine ()
2021-06-25 05:45:20,448 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "AdvStatsW" (
	level_0 TEXT, 
	level_1 TEXT, 
	"Team" TEXT, 
	"Age" BIGINT, 
	"G" BIGINT, 
	"MP" BIGINT, 
	"PER" FLOAT, 
	"TS%" FLOAT, 
	"3PAr" FLOAT, 
	"FTr" FLOAT, 
	"ORB%" FLOAT, 
	"DRB%" FLOAT, 
	"TRB%" FLOAT, 
	"AST%" FLOAT, 
	"STL%" FLOAT, 
	"BLK%" FLOAT, 
	"TOV%" FLOAT, 
	"USG%" FLOAT, 
	"OWS" FLOAT, 
	"DWS" FLOAT, 
	"WS" FLOAT, 
	"WS/48" FLOAT
)


2021-06-25 05:45:20,449 INFO sqlalchemy.engine.base.Engine ()
2021-06-25 05:45:20,451 INFO sqlalchemy.engine.base.Engine COMMIT
2021-06-25 05:45:20,452 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_AdvStatsW_level_1" ON "AdvStatsW" (level_1)
2021-06

In [12]:
sqlite_connection.close()

In [139]:
import sqlite3

WNBA = pd.read_sql('SELECT * FROM WNBAstats;', engine)
WNBA

2021-06-24 14:41:23,843 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM WNBAstats;")
2021-06-24 14:41:23,844 INFO sqlalchemy.engine.base.Engine ()
2021-06-24 14:41:23,846 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM WNBAstats;")
2021-06-24 14:41:23,847 INFO sqlalchemy.engine.base.Engine ()
2021-06-24 14:41:23,849 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM WNBAstats;
2021-06-24 14:41:23,850 INFO sqlalchemy.engine.base.OptionEngine ()


,level_0,level_1,Team,Age,G,MP,PER,TS%,3PAr,FTr,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48
0,Farhiya Abdi,2013,LAS,21,17,122,8.5,0.429,0.000,0.308,...,5.8,11.2,0.8,0.6,8.3,17.9,0.1,0.1,0.2,0.065
1,Farhiya Abdi,2014,LAS,22,17,142,5.7,0.463,0.159,0.068,...,3.9,5.7,0.4,0.0,11.7,16.5,0.0,0.0,0.0,-0.008
2,Farhiya Abdi,2015,LAS,23,18,237,5.4,0.420,0.418,0.089,...,7.8,6.5,1.6,0.7,13.7,19.6,-0.4,0.1,-0.3,-0.055
3,Tajama Abraham,1997,SAC,21,28,422,6.4,0.427,0.000,0.302,...,10.0,6.0,1.5,2.3,25.6,20.3,-0.6,-0.2,-0.8,-0.092
4,Tajama Abraham,1998,DET,22,12,44,8.2,0.437,0.000,1.071,...,9.4,0.0,2.5,1.8,19.5,26.4,-0.1,0.1,0.0,-0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1611,Tayler Hill,2017,WAS,26,18,462,17.0,0.531,0.466,0.419,...,4.2,18.3,1.8,0.2,10.3,24.5,1.3,0.2,1.5,0.155
1612,Tayler Hill,2018,TOT,27,20,235,4.9,0.410,0.536,0.167,...,3.8,12.8,0.7,0.7,15.9,20.3,-0.4,0.0,-0.4,-0.083
1613,Tayler Hill,2018,WAS,27,13,117,6.2,0.440,0.538,0.308,...,4.1,16.6,0.9,0.7,19.9,21.7,-0.2,0.0,-0.2,-0.072
1614,Tayler Hill,2018,DAL,27,7,118,3.7,0.381,0.533,0.044,...,3.4,9.1,0.4,0.7,11.6,18.9,-0.2,0.0,-0.2,-0.094
